In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import random
from typing import TypedDict
from langgraph.graph import END, START, StateGraph
from langgraph.pregel import RetryPolicy

class InputState(TypedDict):
    string_value: str

def modify_state(input: InputState):
    print(f"Current state: {input}")
    if random.random() > 0.7:
        print("Modification succeeded.")
        input["string_value"] += " modified"
    else:
        print("Modification failed, retrying...")
        raise Exception("Simulated error for retry testing")
    return input

graph = StateGraph(InputState)
graph.add_node(
    "branch_a",
    modify_state,
    retry=RetryPolicy(retry_on=Exception, max_attempts=3)
)
graph.add_node(
    "branch_b",
    modify_state,
    retry=RetryPolicy(retry_on=Exception, max_attempts=3)
)
graph.add_edge("branch_a", "branch_b")
graph.add_edge("branch_b", END)
graph.set_entry_point("branch_a")

runnable = graph.compile()

In [ ]:
initial_state = {"string_value": "start"}
result = runnable.invoke(initial_state)
print(f"Final state: {result}")

### Long Term Memory

In [ ]:
from langgraph.store.memory import InMemoryStore

# InMemoryStore saves data to an in-memory dictionary. Use a DB-backed store in production use.
store = InMemoryStore()
user_id = "my-user"
application_context = "chitchat"
namespace = (user_id, application_context)
store.put(namespace, "a-memory", {"rules": ["User likes short, direct language", "User only speaks English & python"], "my-key": "my-value"})
# get the "memory" by ID
item = store.get(namespace, "a-memory")
# list "memories" within this namespace, filtering on content equivalence
items = store.search(namespace, filter={"my-key": "my-value"})